In [6]:
import sys
sys.path.append('..')
import pickle
#harry's code is named "policy"
with open('../brute_force_VI/transposed_brute_force_sol.pkl', 'rb') as f:
    policy_harry = pickle.load(f)

#marks code
with open('../layered_VI/layered_sol.pkl', 'rb') as g:
    policy_mark = pickle.load(g)

with open('../layered_VI/transposed_exact_sol.pkl', 'rb') as h:
    policy_mark_good = pickle.load(h)

FileNotFoundError: [Errno 2] No such file or directory: '../brute_force_VI/transposed_brute_force_sol.pkl'

In [2]:
import random

In [3]:
#trying to simulate a game - policy is going to just be harrys one as he got that working

#dice roll
def roll_die():
    return random.randint(1,6)

def game_pig(policy_player_1, policy_player_2):
    scores = [0,0]
    player = 0 

    while max(scores) < 100:
        turn_total = 0
        while True:
            if (player == 0 and policy_player_1.get((scores[0], scores[1], turn_total), 0)) or \
               (player == 1 and policy_player_2.get((scores[1], scores[0], turn_total), 0)):
                roll = roll_die()
                if roll == 1:
                    turn_total = 0
                    break
                else:
                    turn_total += roll
            else:
                scores[player] += turn_total
                break

        player = 1 - player
    return 0 if scores[0] >= 100 else 1 
    

In [4]:

#multiple simulations
def simulate_games(n, policy1, policy2):
    results = [0,0]
    for _ in range(n):
        winner = game_pig(policy1, policy2)
        results[winner] +=1
    return results


In [5]:
results_mark = simulate_games(1000000, policy_mark_good, policy_mark_good)
print(f"Player 0 wins: {results_mark[0]}, Player 1 wins: {results_mark[1]}")
win_rate = results_mark[0]/1000000
print(win_rate) #got 0.530412 - this lines up with the paper's value 0.5306

NameError: name 'policy_mark_good' is not defined

In [487]:
results_harry = simulate_games(100000, policy_harry, policy_harry)
print(f"Player 0 wins: {results_harry[0]}, Player 1 wins: {results_harry[1]}")

Player 0 wins: 53241, Player 1 wins: 46759


In [488]:
results_mark = simulate_games(10000, policy_harry, policy_mark_good)
print(f"Player 0 wins: {results_mark[0]}, Player 1 wins: {results_mark[1]}")

Player 0 wins: 5306, Player 1 wins: 4694


In [489]:
results_mark = simulate_games(10000, policy_mark_good, policy_harry)
print(f"Player 0 wins: {results_mark[0]}, Player 1 wins: {results_mark[1]}")

Player 0 wins: 5290, Player 1 wins: 4710


In [490]:
#sophies policy from the pig competition

policy_sophie = {
    (my_score, op_score, round_score): (
        False if my_score + round_score > 100
        else round_score < 20
             or my_score >= 71
    )
    for my_score in range(101)
    for op_score in range(101)
    for round_score in range(101)
}


In [491]:
#hold at 20 policy

policy_hold_at_20 = {
    (my_score, op_score, round_score): (
        False if round_score >= 20 or my_score + round_score > 100 else True
    )
    for my_score in range(101)
    for op_score in range(101)
    for round_score in range(101)
}

In [492]:
# running optimal against sophies policy
# paper says expected result should be optimal winning 58.74% of the time

results_mark_v_sophie = simulate_games(100000, policy_mark_good, policy_sophie)
print(f"Player 0 wins: {results_mark_v_sophie[0]}, Player 1 wins: {results_mark_v_sophie[1]}")


Player 0 wins: 57673, Player 1 wins: 42327


In [498]:
#running sophie vs mark (so hold at 20 unless very close to goal goes first)
#paper says p1 should win 47.76

results_sophie_v_mark = simulate_games(100000, policy_sophie, policy_mark_good)
print(f"Player 0 wins: {results_sophie_v_mark[0]}, Player 1 wins: {results_sophie_v_mark[1]}")

Player 0 wins: 48592, Player 1 wins: 51408


In [1]:
#optimal vs hold at 20
#optimal should win more than 58% of the time i think - error in paper
import statistics
import math
random.seed(123)
results_mark_v_20 = simulate_games(100000, policy_mark_good, policy_hold_at_20)
print(f"Player 0 wins: {results_mark_v_20[0]}, Player 1 wins: {results_mark_v_20[1]}")
win_rate1 = results_mark_v_20[0]/100000
raw_res = [0]*results_mark_v_20[0]
raw_res.extend([1]*results_mark_v_20[1])
CI_low = win_rate1 - 2.58*math.sqrt(statistics.variance(raw_res))/100000
CI_high = win_rate1 + 2.58*math.sqrt(statistics.variance(raw_res))/100000
print(win_rate1)
print([CI_low, CI_high])
2.5

NameError: name 'random' is not defined

In [30]:
list1 = [1]*1000
list1.extend([0]*100)
statistics.variance(list1)

0.08271982794275788

In [ ]:
#hold at 20 vs optimal - error in paper

results_20_v_mark = simulate_games(100000, policy_hold_at_20, policy_mark_good)
print(f"Player 0 wins: {results_20_v_mark[0]}, Player 1 wins: {results_20_v_mark[1]}")
win_rate2 = results_20_v_mark[0]/100000
raw_res2 = [0]*results_20_v_mark[0]
raw_res2.extend([1]*results_20_v_mark[1])
CI_low2 = win_rate2 - 2.58*math.sqrt(statistics.variance(raw_res2)/100000)
CI_high2 = win_rate2 + 2.58*math.sqrt(statistics.variance(raw_res2)/100000)
print(win_rate2)
print([CI_low2, CI_high2])

NameError: name 'policy_hold_at_20' is not defined